# Get data and packages

## Prep benchmarks

In [5]:
import vaex
import pandas as pd
import numpy as np
import warnings
import datetime as dt
import time
import gc

warnings.filterwarnings("ignore")

benchmarks = {}

def benchmark(f, name, df, repetitions=1, **kwargs):
    times = []
    count = 0
    for i in range(repetitions):
        start_time = time.time()
        ret = f(df, **kwargs)
        times.append(time.time()-start_time)
        count+=1
    benchmarks[name] = np.mean(times)/count
    return benchmarks[name]

def get_results(benchmarks, name):
    results = pd.DataFrame.from_dict(benchmarks, orient='index')
    results.columns = [name]
    return results

### Vaex implementation

In [6]:
def read_file():
    return vaex.open('datasets/taxi_1m.hdf5')
    
def mean(df):
    return df.fare_amount.mean()
    
def standard_deviation(df):
    return df.fare_amount.std()

def sum_columns(df):
    return df.fare_amount + df.passenger_count

def product_columns(df):
    return df.fare_amount * df.passenger_count

def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    return 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))

def value_counts(df):
    return df.passenger_count.value_counts()

def groupby_statistics(df):
    return df.groupby(by='pickup_hour').agg({'fare_amount': ['mean', 'std'], 
                                               'tip_amount': ['mean', 'std']
                                              })
def join(df, other):
    df_joined = df.join(other=gp, on = 'pickup_hour', rsuffix = '_right')
    

def filter_data(df):
    return df[expr_filter]



## Naive

In [10]:
# Load data
data = vaex.open('datasets/taxi_1m.hdf5')
data['pickup_hour'] = data.pickup_datetime.dt.hour

In [11]:
# benchmark
benchmark(mean, 'mean', data, repetitions=10)
benchmark(standard_deviation,'standard deviation', data, repetitions=10)
benchmark(sum_columns, 'sum columns', data, repetitions=10)
benchmark(product_columns, 'product columns', data, repetitions=10)
benchmark(complicated_arithmetic_operation, 'complicated arithmetic operation', data, repetitions=10)
benchmark(value_counts, 'value counts', data, repetitions=10)
benchmark(groupby_statistics, 'groupby statistics', data, repetitions=10)

gp = data.groupby(by='pickup_hour').agg({'fare_amount': ['mean', 'std'], 
                                        'tip_amount': ['mean', 'std']
                                        })
benchmark(join, 'join', data, repetitions=10, other=gp)
print(f"Done benchmarks on all data")

Done benchmarks on all data


## Filtered

In [12]:
# load data
long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90

expr_filter = (data.pickup_longitude > long_min)  & (data.pickup_longitude < long_max) & \
              (data.pickup_latitude > lat_min)    & (data.pickup_latitude < lat_max) & \
              (data.dropoff_longitude > long_min) & (data.dropoff_longitude < long_max) & \
              (data.dropoff_latitude > lat_min)   & (data.dropoff_latitude < lat_max)

filterd = data[expr_filter]
del data
del gp

deleted = gc.collect()
print(f"Prepare filtered data and deleted {deleted} MB")

Prepare filtered data and deleted 1864 MB


In [13]:
# benchmark
benchmark(mean, 'filtered mean', filterd, repetitions=10)
benchmark(standard_deviation,'filtered standard deviation', filterd, repetitions=10)
benchmark(sum_columns, 'filtered sum columns', filterd, repetitions=10)
benchmark(product_columns, 'filtered product_columns', filterd, repetitions=10)
benchmark(complicated_arithmetic_operation, 'filtered complicated arithmetic_operation', filterd, repetitions=10)
benchmark(value_counts, 'filtered value_counts', filterd, repetitions=10)
benchmark(groupby_statistics, 'filtered groupby statistics', filterd, repetitions=10)

gp = filterd.groupby(by='pickup_hour').agg({'fare_amount': ['mean', 'std'], 
                                        'tip_amount': ['mean', 'std']
                                        })
benchmark(join, 'filtered join', filterd, repetitions=10, other=gp)

0.006154913902282714

In [17]:
name = 'vaex'
results = get_results(benchmarks, name)
results.to_csv(f"results/{name}_1m.csv")
results.head()

,vaex
mean,9.630513e-04
standard deviation,2.164185e-03
sum columns,1.227856e-06
product columns,8.893013e-07
complicated arithmetic operation,4.458714e-04
